<a href="https://colab.research.google.com/github/polyphemus287/IBM-Capstone/blob/main/capstone_plotly_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install jupyter-dash

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# Create options for dropdown
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]


# Create a dash application
app = dash.Dash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                       style={'textAlign': 'center', 'color': '#503D36',
                                              'font-size': 40}),
                               # Add a dropdown list to enable Launch Site selection
                               dcc.Dropdown(id='site-dropdown',
                                            options=launch_sites,
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True),
                               html.Br(),


                               # Add a pie chart to show the total successful launches count for all sites
                               html.Div(dcc.Graph(id='success-pie-chart')),
                               html.Br(),


                               html.P("Payload range (Kg):"),
                               # Add a slider to select payload range
                               dcc.RangeSlider(id='payload-slider',
                                               min=0, max=10000, step=1000,
                                               marks={0: '0', 10000: '10000'},
                                               value=[min_payload, max_payload]
                                               ),


                               # Add a scatter chart to show the correlation between payload and launch success
                               html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                               ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart', 'figure'),
             Input('site-dropdown', 'value'))
def update_pie_chart(selected_site):
   if selected_site == 'ALL':
       fig = px.pie(spacex_df, names='class', title='Total Success Launches')
   else:
       filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
       fig = px.pie(filtered_df, names='class', title=f'Success Launches at {selected_site}')
   return fig


# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart', 'figure'),
             [Input('site-dropdown', 'value'),
              Input('payload-slider', 'value')])
def update_scatter_chart(selected_site, payload_range):
   low, high = payload_range
   filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
   if selected_site == 'ALL':
       title = f'Payload Success Correlation'
   else:
       filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
       title = f'Payload Success Correlation at {selected_site}'
   fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=title)
   return fig


# Run the app
if __name__ == '__main__':
   app.run_server()

<IPython.core.display.Javascript object>